In [3]:
%matplotlib
import pandas as pd
import numpy as np
import torch
from torch import nn

Using matplotlib backend: <object object at 0x7fd81757b340>


In [5]:
train_data = pd.read_csv('../house-prices/train.csv')
test_data = pd.read_csv('../house-prices/test.csv')
print(train_data)
print(test_data)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities  ... PoolArea PoolQC  Fe

In [9]:
print(train_data.iloc[0:4, [0,1,2,3,-3,-2,-1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [19]:
all_features = pd.concat([train_data.iloc[:,1:-1],test_data.iloc[:,1:]])
print(all_features)

      MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0             60       RL         65.0     8450   Pave   NaN      Reg   
1             20       RL         80.0     9600   Pave   NaN      Reg   
2             60       RL         68.0    11250   Pave   NaN      IR1   
3             70       RL         60.0     9550   Pave   NaN      IR1   
4             60       RL         84.0    14260   Pave   NaN      IR1   
...          ...      ...          ...      ...    ...   ...      ...   
1454         160       RM         21.0     1936   Pave   NaN      Reg   
1455         160       RM         21.0     1894   Pave   NaN      Reg   
1456          20       RL        160.0    20000   Pave   NaN      Reg   
1457          85       RL         62.0    10441   Pave   NaN      Reg   
1458          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC  Fence  \
0            Lvl    AllPub    Inside  ...  

In [20]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x : (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)
print(all_features) 

      MSSubClass MSZoning  LotFrontage   LotArea Street Alley LotShape  \
0       0.067320       RL    -0.184443 -0.217841   Pave   NaN      Reg   
1      -0.873466       RL     0.458096 -0.072032   Pave   NaN      Reg   
2       0.067320       RL    -0.055935  0.137173   Pave   NaN      IR1   
3       0.302516       RL    -0.398622 -0.078371   Pave   NaN      IR1   
4       0.067320       RL     0.629439  0.518814   Pave   NaN      IR1   
...          ...      ...          ...       ...    ...   ...      ...   
1454    2.419286       RM    -2.069222 -1.043758   Pave   NaN      Reg   
1455    2.419286       RM    -2.069222 -1.049083   Pave   NaN      Reg   
1456   -0.873466       RL     3.884968  1.246594   Pave   NaN      Reg   
1457    0.655311       RL    -0.312950  0.034599   Pave   NaN      Reg   
1458    0.067320       RL     0.201080 -0.068608   Pave   NaN      Reg   

     LandContour Utilities LotConfig  ... ScreenPorch  PoolArea PoolQC  Fence  \
0            Lvl    AllPub    

In [27]:
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.values

array([[0.06731987999872384, -0.1844426168571562, -0.21784136849269334,
        ..., True, False, False],
       [-0.8734663783676423, 0.4580958808711674, -0.07203174114674873,
        ..., True, False, False],
       [0.06731987999872384, -0.05593491731149148, 0.13717337634960655,
        ..., True, False, False],
       ...,
       [-0.8734663783676423, 3.8849678687555604, 1.2465944539817937, ...,
        False, False, False],
       [0.6553112914777027, -0.31295031640282095, 0.034599473286242056,
        ..., True, False, False],
       [0.06731987999872384, 0.20108048177983798, -0.06860838467862655,
        ..., True, False, False]], dtype=object)

In [39]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values.astype(float), dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values.astype(float), dtype=torch.float32)
train_lables = torch.tensor(train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)

In [41]:
train_features

tensor([[ 0.0673, -0.1844, -0.2178,  ...,  1.0000,  0.0000,  0.0000],
        [-0.8735,  0.4581, -0.0720,  ...,  1.0000,  0.0000,  0.0000],
        [ 0.0673, -0.0559,  0.1372,  ...,  1.0000,  0.0000,  0.0000],
        ...,
        [ 0.3025, -0.1416, -0.1428,  ...,  1.0000,  0.0000,  0.0000],
        [-0.8735, -0.0559, -0.0572,  ...,  1.0000,  0.0000,  0.0000],
        [-0.8735,  0.2439, -0.0293,  ...,  1.0000,  0.0000,  0.0000]])

In [ ]:
loss = nn.MSELoss()
in_features = train_features.shape[1]

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

def log_rmse(net, features, labels):
    